In [3]:
#import and install packages 
import sys
import re 
import numpy as np 
import pandas as pd 
from pprint import pprint
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('/Users/maximascientia/Documents/Reviews.csv')
df.head

<bound method NDFrame.head of        Unnamed: 0  Clothing ID  Age  \
0               0          767   33   
1               1         1080   34   
2               2         1077   60   
3               3         1049   50   
4               4          847   47   
...           ...          ...  ...   
23481       23481         1104   34   
23482       23482          862   48   
23483       23483         1104   31   
23484       23484         1084   28   
23485       23485         1104   52   

                                                   Title  \
0                                                    NaN   
1                                                    NaN   
2                                Some major design flaws   
3                                       My favorite buy!   
4                                       Flattering shirt   
...                                                  ...   
23481                     Great dress for many occasions   
23482                

In [22]:
#Create our lists of stopwords
stopwords = set('for a of the and an to in from am are is was were be'.split(' '))
#Put each document in lowercase and separate with spaces
texts = [[word for word in df.lower().split() if word not in stopwords]
        for df in df]

#Count the frequency of words
from collections import defaultdict
frequency = defaultdict(int)
for text in texts: 
    for token in text: 
        frequency[token] +=1

#We want to keep words that are found more than once 
processed_df = [[token for token in text if frequency[token] > 1] for text in texts]

In [28]:
#This removes random stuff that may be in the code.
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent) 

# Convert Comments to list
df['ReviewText'] = df['ReviewText'].apply(str)
data = df.ReviewText.values.tolist()
data_words = list((data))
print(data_words[:1])

['Absolutely wonderful - silky and sexy and comfortable']


<>:4: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
/var/folders/8t/_9gdklfd5j9bftg20qg_bn0m0000gn/T/ipykernel_48853/1784516939.py:4: DeprecationWarning: invalid escape sequence \S
  sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
/var/folders/8t/_9gdklfd5j9bftg20qg_bn0m0000gn/T/ipykernel_48853/1784516939.py:5: DeprecationWarning: invalid escape sequence \s
  sent = re.sub('\s+', ' ', sent)  # remove newline chars


In [29]:
# remove stop words and words that appear only once
stop_ids = [
    dictionary.token2id[stopword]
    for stopword in stopwords
    if stopword in dictionary.token2id
]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)

Dictionary(15983 unique tokens: ['-', 'absolutely', 'comfortable', 'sexy', 'silky']...)


In [30]:
#collect statistics about all tokens and create the corpa
id2word = corpora.Dictionary(line.lower().split() for line in data_words)
corpus = [dictionary.doc2bow(text) for text in texts]

In [31]:
#Initialize Model
from gensim import models 
tfidf = models.TfidfModel(corpus)

In [32]:
#Transform the whole model into a vector 
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(807, 1.0)]
[(1560, 0.7071067811865476), (5279, 0.7071067811865476)]
[(2339, 1.0)]
[(1946, 1.0)]
[]
[(7304, 1.0)]
[(1205, 1.0)]
[(1941, 0.5773502691896258), (8131, 0.5773502691896258), (10672, 0.5773502691896258)]
[(4019, 1.0)]
[(4019, 0.4764030728504269), (8391, 0.8792269969573677)]
[(3398, 0.8792269969573677), (4019, 0.4764030728504269)]


In [39]:
model = models.LdaModel(corpus, id2word=id2word, num_topics=4)

/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


In [40]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, corpus, dictionary=model.id2word)
vis

/Users/maximascientia/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.000101 -0.0       1        1  31.240282
2     -0.000075 -0.0       2        1  28.527276
1     -0.000039  0.0       3        1  27.297879
3      0.000012  0.0       4        1  12.934563, topic_info=                Term      Freq     Total Category  logprob  loglift
4019            air.  0.000000  0.000000  Default  30.0000  30.0000
2339         climate  0.000000  0.000000  Default  29.0000  29.0000
5279           sorry  0.000000  0.000000  Default  28.0000  28.0000
3398         putting  0.000000  0.000000  Default  27.0000  27.0000
1560       limitless  0.000000  0.000000  Default  26.0000  26.0000
...              ...       ...       ...      ...      ...      ...
5279           sorry  0.000053  0.000826   Topic4 -10.5076  -0.7011
8131   conditioning!  0.000053  0.000886   Topic4 -10.5053  -0.7686
1560       limitless  0.000053  0.000826   Topic4 -10.5105  -0.7038
10672             (1  0.000053  0.000887   Topic4 -10.5144  -0.7791
1941           nice,  0.000053  0.000887   Topic4 -10.5166  -0.7806

[188 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4])